<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Mathematics Basics

**With `NumPy` and `SciPy`**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Portfolio Optimization

From Wikipedia (https://en.wikipedia.org/wiki/Portfolio_optimization):

> Portfolio optimization is the process of selecting the best portfolio (asset distribution), out of the set of all portfolios being considered, according to some objective. The objective typically maximizes factors such as expected return, and minimizes costs like financial risk.

## Modern Portfolio Theory

From Wikipedia (https://en.wikipedia.org/wiki/Portfolio_optimization):

> Modern portfolio theory was introduced in a 1952 doctoral thesis by Harry Markowitz; see Markowitz model. It assumes that an investor wants to maximize a portfolio's expected return contingent on any given amount of risk. For portfolios that meet this criterion, known as efficient portfolios, achieving a higher expected return requires taking on more risk, so investors are faced with a trade-off between risk and expected return. This risk-expected return relationship of efficient portfolios is graphically represented by a curve known as the efficient frontier. All efficient portfolios, each represented by a point on the efficient frontier, are well-diversified.

The seminal paper on the topic is found e.g. under [Markowitz (1952)](https://www.math.hkust.edu.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf).

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/mathematics_basics.git
import sys
sys.path.append('mathematics_basics')


In [ ]:
import numpy as np
np.set_printoptions(suppress=True)

In [ ]:
from pylab import plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

For `minimize` see https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html.

In [ ]:
from scipy.optimize import minimize

## Importing Data

In [ ]:
import urllib

### The File

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
fn = '/content/'+url.split('/')[-1]

In [ ]:
fn

In [ ]:
urllib.request.urlretrieve(url, fn)

### Reading from Disk

In [ ]:
# np.genfromtxt?

In [ ]:
raw = np.genfromtxt(fn, delimiter=',', names=True)

In [ ]:
raw = raw[list(raw.dtype.names)[1:]]

In [ ]:
raw[:3]

### Dropping `NaN` Values

In [ ]:
np.isnan(raw['SPX'])

In [ ]:
~np.isnan(raw['SPX'])

In [ ]:
raw = raw[~np.isnan(raw['SPX'])]

In [ ]:
raw[:3]

In [ ]:
plt.plot(raw['AAPLO']);

## Asset Returns

In [ ]:
raw.tolist()[:1]

In [ ]:
data = np.array(raw.tolist())

In [ ]:
data[:5, :4]

In [ ]:
rets_ = np.log(data[1:] / data[:-1])

In [ ]:
rets_[:5, :4]

In [ ]:
plt.hist(rets_[:, 0], bins=50);  # histogram of log returns of AAPL.O

In [ ]:
noa = 2

In [ ]:
rets = rets_[:, :noa]

In [ ]:
rets

## Asset Statistics

In [ ]:
import math

In [ ]:
rets.mean(axis=0) * 252  # annualized return

In [ ]:
rets.std(axis=0) * math.sqrt(252)  # annualized volatility

## Portfolio Statistics

In [ ]:
phi = noa * [1 / noa]  # equally weighted portfolio

In [ ]:
phi

In [ ]:
def portfolio_return(phi):
    return np.dot(rets.mean(axis=0), phi) * 252  # annualized

In [ ]:
portfolio_return(phi)

In [ ]:
np.cov(rets.T)

In [ ]:
def portfolio_volatility(phi):
    return math.sqrt(np.dot(phi, np.dot(np.cov(rets.T), phi)) * 252)  # annualized

In [ ]:
portfolio_volatility(phi)

## Random Portfolios

In [ ]:
w = np.random.random((1000, noa))

In [ ]:
w[:3]

In [ ]:
w = (w.T / w.sum(axis=1)).T  # normalization

In [ ]:
w[:3]

In [ ]:
w.sum(axis=1)[:3]

In [ ]:
rr = [(portfolio_volatility(phi), portfolio_return(phi)) for phi in w]

In [ ]:
rr = np.array(rr)

In [ ]:
rr[:3]

In [ ]:
plt.plot(rr[:, 0], rr[:, 1], 'r.')
plt.xlabel('portfolio risk')
plt.ylabel('portfolio return');

## Minimum Risk Portfolio

In [ ]:
bnds = noa * [(0, 1),]

In [ ]:
cons = {'type': 'eq', 'fun': lambda phi: phi.sum() - 1}

In [ ]:
opt = minimize(portfolio_volatility, phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
mrp = portfolio_volatility(opt['x']), portfolio_return(opt['x'])

In [ ]:
mrp

In [ ]:
plt.plot(rr[:, 0], rr[:, 1], 'r.')
plt.plot(mrp[0], mrp[1], 'bo', label='MRP')
plt.legend();

## Maximum Sharpe Ratio Portfolio

In [ ]:
def sharpe_ratio(phi):
    return portfolio_return(phi) / portfolio_volatility(phi)

In [ ]:
sharpe_ratio(phi)

In [ ]:
opt = minimize(lambda phi: -sharpe_ratio(phi),
               phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
msp = portfolio_volatility(opt['x']), portfolio_return(opt['x'])

In [ ]:
msp

In [ ]:
sharpe_ratio(opt['x'])

In [ ]:
plt.plot(rr[:, 0], rr[:, 1], 'r.')
plt.plot(mrp[0], mrp[1], 'bo', label='MRP')
plt.plot(msp[0], msp[1], 'go', label='MSP')
plt.legend();

<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>